# Data Collection

---



In [3]:
import pandas as pd
test_data_df = pd.read_csv('test_data.csv')
train_data_df = pd.read_csv('train_data.csv')
title_brand_df = pd.read_csv('title_brand.csv')

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Import Data

import gdown
import pandas as pd

# Train Data
url = 'https://drive.google.com/uc?id=1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW'
output = 'train_data.csv'
gdown.download(url, output, quiet=False)
train_data_df = pd.read_csv('train_data.csv')

# Test Data
url = 'https://drive.google.com/uc?id=1-8TsrqTRFP-q9TM-6HinhO0ZVXFHq9TB'
output = 'test_data.csv'
gdown.download(url, output, quiet=False)
test_data_df = pd.read_csv('test_data.csv')

# Product Identifier Mapping Table by their Title and Brand
url = 'https://drive.google.com/uc?id=1I9aPAvvYgQWdHGKtnd7IeTGXpx8vOm4h'
output = 'title_brand.csv'
gdown.download(url, output, quiet=False)
title_brand_df = pd.read_csv('title_brand.csv')

In [5]:
train_data_df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,2,NaN,False,2016-11-11,A2OSUEZJIN7BI,0511189877,NaN,Chris,I have an older URC-WR7 remote and thought thi...,Cannot Learn,1478822400
1,5,NaN,True,2016-06-06,A2NETQRG6JHIG7,0511189877,NaN,Qrysta White,First time I've EVER had a remote that needed ...,zero programming needed! Miracle!?,1465171200
2,4,NaN,True,2016-03-10,A12JHGROAX49G7,0511189877,NaN,Linwood,Got them and only 2 of them worked. company ca...,Works Good and programs easy.,1457568000
3,5,NaN,True,2016-01-14,A1KV65E2TMMG6F,0511189877,NaN,Dane Williams,I got tired of the remote being on the wrong s...,Same as TWC remote,1452729600
4,5,NaN,True,2016-10-20,A280POPEWI0NSA,0594459451,NaN,Kristina H.,After purchasing cheap cords from another webs...,Good Quality Cord,1476921600


In [6]:
test_data_df.head()

,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,NaN,True,2018-06-04,A20GGWE66JW9X2,B006Z394GM,{'Color:': ' FPS01-C'},Brian C Toner,The name and description of this device are mi...,The prize for most useless invention of all ti...,1528070400
1,NaN,True,2018-02-18,ARYJWXHEYHG9M,B005BE058W,"{'Size:': ' 1000W', 'Style:': ' G2'}",Snake,One of the molex connectors on the power suppl...,MELTED MOLEX CONNECTOR,1518912000
2,NaN,True,2018-01-20,A10LHZ7WFZ7HLL,B01DA0YCNC,NaN,Amazon Customer,Remote constantly disconnects/ Roku player fre...,Bricked on the regular,1516406400
3,NaN,True,2018-06-25,A11VN8EOHNLP72,B00FBJ4KYC,NaN,Jeremy Bray,I purchased this 4 year protection plan for a ...,DO NOT BUY!!!,1529884800
4,3.0,True,2016-08-17,A194Y8P8TVT7P9,B00P7G82TS,NaN,Mark,I bought one of these and have regretted it ev...,Nightmare - don't buy,1471392000


In [7]:
title_brand_df.head()

,asin,title,brand
0,0011300000,Genuine Geovision 1 Channel 3rd Party NVR IP S...,GeoVision
1,0043396828,"Books ""Handbook of Astronomical Image Processi...",33 Books Co.
2,0060009810,One Hot Summer,Visit Amazon's Carolina Garcia Aguilera Page
3,0060219602,Hurray for Hattie Rabbit: Story and pictures (...,Visit Amazon's Dick Gackenbach Page
4,0060786817,sex.lies.murder.fame.: A Novel,Visit Amazon's Lolita Files Page


# Sentiment analysis model

---



## 1: Using VADER SentimentIntensityAnalyser to calculate Sentiment Score

<p>VADER is a rule-based sentiment analysis tool specifically designed for social media text, including internet reviews. It uses a combination of lexical and grammatical heuristics to determine sentiment polarity (positive, negative, or neutral) and also provides a sentiment intensity score. VADER has been trained on a vast amount of data and is known for its effectiveness in handling informal language, slang, and context-specific sentiment expressions.

It is not recommended to use general text preprocessing techniques prior to calculation because this may affect VADER results.</p>

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

# Use the first 20,000 rows
df_VADER = train_data_df[['overall', 'summary', 'reviewText']].copy().iloc[:200000]

df_VADER['text_and_summary'] = df_VADER['summary'] + " " + df_VADER['reviewText']
df_VADER['text_and_summary'] = df_VADER['text_and_summary'].fillna('')

sent = SentimentIntensityAnalyzer()
polarity = [round(sent.polarity_scores(i)['compound'], 2) for i in df_VADER['text_and_summary']]
df_VADER['sentiment_score'] = polarity

def sentiment_to_star_rating(sentiment_score):
    star_rating = 2 * sentiment_score + 3
    star_rating = round(star_rating)
    return star_rating

df_VADER['star_rating'] = df_VADER['sentiment_score'].apply(sentiment_to_star_rating)
df_VADER.head()



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,overall,summary,reviewText,text_and_summary,sentiment_score,star_rating
0,2,Cannot Learn,I have an older URC-WR7 remote and thought thi...,Cannot Learn I have an older URC-WR7 remote an...,-0.94,1
1,5,zero programming needed! Miracle!?,First time I've EVER had a remote that needed ...,zero programming needed! Miracle!? First time...,0.95,5
2,4,Works Good and programs easy.,Got them and only 2 of them worked. company ca...,Works Good and programs easy. Got them and onl...,0.84,5
3,5,Same as TWC remote,I got tired of the remote being on the wrong s...,Same as TWC remote I got tired of the remote b...,-0.72,2
4,5,Good Quality Cord,After purchasing cheap cords from another webs...,Good Quality Cord After purchasing cheap cords...,0.82,5


In [ ]:
from sklearn.metrics import f1_score

df_VADER['overall'] = df_VADER['overall'].astype(int)
df_VADER['star_rating'] = df_VADER['star_rating'].astype(int)

f1 = f1_score(df_VADER['overall'], df_VADER['star_rating'], average='micro')
print(f"The F1 score is {f1}")


The F1 score is 0.517995


## 2 : Bidirectional LSTM Model with Pre-trained Word Embeddings - Classification

### Random Undersampling Data

In [8]:
import numpy as np
import os
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM, Dropout, Bidirectional, Conv1D, MaxPooling1D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

random.seed(42)


df = train_data_df
df["text"] = df.apply(lambda x: str(x["summary"]) + " " + str(x["reviewText"]), axis=1)
df_train, df_val = train_test_split(df, test_size=0.1, stratify=df['overall'], random_state=42)

# Balancing df_train

under = RandomUnderSampler(sampling_strategy={1: 50000, 2: 50000, 3: 50000, 4: 50000, 5: 50000})
df_train_balanced, y_over = under.fit_resample(df_train, df_train['overall'])


# Display new class counts
print(df_train_balanced.overall.value_counts())

overall
1    50000
2    50000
3    50000
4    50000
5    50000
Name: count, dtype: int64


### Tokenization and One Hot Encoding

In [9]:
tokenizer=Tokenizer(oov_token="'oov'")
tokenizer.fit_on_texts(df_train_balanced['text'])
maxlen = 200

train_X = pad_sequences(tokenizer.texts_to_sequences(df_train_balanced['text']), maxlen=maxlen)
val_X = pad_sequences(tokenizer.texts_to_sequences(df_val['text']), maxlen=maxlen)

# One-hot encode the 'overall' column
encoder = OneHotEncoder(sparse=False)
train_Y = encoder.fit_transform(df_train_balanced["overall"].values.reshape(-1, 1))
val_Y = encoder.transform(df_val["overall"].values.reshape(-1, 1))

### Loading GloVe pre-trained word embeddings and preparing an embedding matrix for neural network model

<p>source: https://www.kaggle.com/datasets/rtatman/glove-global-vectors-for-word-representation</p>

In [10]:
glove_dir="/Users/sarayetel/Desktop/Data Science/Project2/Part 3"
embedding_index = {}
f = open(os.path.join(glove_dir,'glove.6B.100d.txt'),encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedding_index[word] = coefs
f.close()
print('Found %s word vectors ' % len(embedding_index))

max_words = len(tokenizer.word_index) + 1
embedding_dim = 100
embedding_matrix = np.zeros((max_words,embedding_dim))

for word, idx in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx]=embedding_vector

Found 400000 word vectors 


## Classification

<p>The following model was used for classification, which consists of a non-trainable embedding layer on the input, a bidirectional LSTM layer, a dense layer with ReLU, and an output layer with the softmax function over 5 classes. As expected, the training of the model was done using the cross-entropy loss function.</p>

In [11]:
from sklearn.metrics import f1_score

model=Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(16, activation="relu"))
model.add(Dense(5, activation="softmax")) 
model.compile(optimizer="Adam", loss='categorical_crossentropy', metrics=['accuracy']) 
print(model.summary())
model.fit(train_X, train_Y, epochs=30, batch_size=256, validation_data=(val_X, val_Y))


pred = model.predict(val_X)
pred_hard = np.argmax(pred, axis=1) + 1


f1 = f1_score(np.argmax(val_Y, axis=1) + 1, pred_hard, average='micro')
print(f'F1 Score (Micro): {f1}')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          13773600  
                                                                 
 bidirectional (Bidirection  (None, 64)                34048     
 al)                                                             
                                                                 
 dense (Dense)               (None, 16)                1040      
                                                                 
 dense_1 (Dense)             (None, 5)                 85        
                                                                 
Total params: 13808773 (52.68 MB)
Trainable params: 35173 (137.39 KB)
Non-trainable params: 13773600 (52.54 MB)
_________________________________________________________________
None
Epoch 1/30
977/977 [==============================] - 195s 197ms/step -

### Test Data Prediction

In [12]:
test_data_df["text"] = test_data_df.apply(lambda x: str(x["summary"]) + " " + str(x["reviewText"]), axis=1)
test_X = pad_sequences(tokenizer.texts_to_sequences(test_data_df['text']), maxlen=maxlen)

In [13]:
test_pred = model.predict(test_X)
test_pred_hard = np.argmax(test_pred, axis=1) + 1

625/625 [==============================] - 7s 11ms/step


In [17]:
# Save the DataFrame to a csv file
submission_df = pd.DataFrame(test_pred_hard, columns=['predicted'])
submission_df.to_csv('q2_submission.csv', index=False)